In [ ]:
import open3d as o3d
from torch_cluster import fps
import torch
import os,sys
import numpy as np

## check the dictionary

In [2]:
path='tmp/scene0000_00_vh_clean_2.pth'
data=torch.load(path)
print(data.keys())

dict_keys(['coords', 'feats', 'sem_label', '1024_coords', '2048_coords', '4096_coords', '1024_feats', '2048_feats', '4096_feats', '1024_sem_label', '2048_sem_label', '4096_sem_label', 'scene_name', 'scene_label', '256_coords', '256_feats', '256_sem_label', '512_coords', '512_feats', '512_sem_label', '128_coords', '128_feats', '128_sem_label', '32_coords', '32_feats', '32_sem_label', '64_coords', '64_feats', '64_sem_label', '8192_coords', '8192_feats', '8192_sem_label', '16184_coords', '16184_feats', '16184_sem_label', '1_coords', '1_feats', '1_sem_label', '0_coords', '0_feats', '0_sem_label'])


In [7]:
for key in data.keys():
    print(key,type(data[key]))

coords <class 'numpy.ndarray'>
feats <class 'numpy.ndarray'>
sem_label <class 'numpy.ndarray'>
1024_coords <class 'numpy.ndarray'>
2048_coords <class 'numpy.ndarray'>
4096_coords <class 'numpy.ndarray'>
1024_feats <class 'numpy.ndarray'>
2048_feats <class 'numpy.ndarray'>
4096_feats <class 'numpy.ndarray'>
1024_sem_label <class 'numpy.ndarray'>
2048_sem_label <class 'numpy.ndarray'>
4096_sem_label <class 'numpy.ndarray'>
scene_name <class 'str'>
scene_label <class 'int'>
256_coords <class 'numpy.ndarray'>
256_feats <class 'numpy.ndarray'>
256_sem_label <class 'numpy.ndarray'>
512_coords <class 'numpy.ndarray'>
512_feats <class 'numpy.ndarray'>
512_sem_label <class 'numpy.ndarray'>
128_coords <class 'numpy.ndarray'>
128_feats <class 'numpy.ndarray'>
128_sem_label <class 'numpy.ndarray'>
32_coords <class 'numpy.ndarray'>
32_feats <class 'numpy.ndarray'>
32_sem_label <class 'numpy.ndarray'>
64_coords <class 'numpy.ndarray'>
64_feats <class 'numpy.ndarray'>
64_sem_label <class 'numpy.ndarr

In [9]:
dtype=torch.float
device=torch.device('cuda:0')
def mFPS(in_path,out_path,expect_num,device,dtype):
    pcd=o3d.io.read_point_cloud(in_path)
    dense_pts=torch.tensor(pcd.points,dtype=dtype,device=device)
    n_pts=dense_pts.size()[0]
    ratio=expect_num/n_pts*1.0
    sparse_indice=fps(dense_pts,ratio=ratio,random_start=False)
    if(sparse_indice.size()[0]!=expect_num):
        sparse_indice=sparse_indice[list(np.linspace(0,sparse_indice.size()[0],expect_num,endpoint=False,dtype=int))]
    sparse_indice=sparse_indice.tolist()
    assert len(sparse_indice)==expect_num
    
    # sparse pcd
    sparse_points=np.asarray(pcd.points)[sparse_indice]
    sparse_colors=np.asarray(pcd.colors)[sparse_indice]
    pcd.points=o3d.utility.Vector3dVector()
    pcd.colors=o3d.utility.Vector3dVector()
    pcd.points=o3d.utility.Vector3dVector(sparse_points)
    pcd.colors=o3d.utility.Vector3dVector(sparse_colors)
    
    # write
    assert o3d.io.write_point_cloud(out_path, pcd)==True

# General ScanNet info

- **Reconstructed surface mesh file (`*.ply`)**:
Binary PLY format mesh with +Z axis in upright orientation.

- **RGB-D sensor stream (`*.sens`)**:
Compressed binary format with per-frame color, depth, camera pose and other data.  See [ScanNet C++ Toolkit](#scannet-c-toolkit) for more information and parsing code. See [SensReader/python](SensReader/python) for a very basic python data exporter.

- **Surface mesh segmentation file (`*.segs.json`)**:
```javascript
{
  "params": {  // segmentation parameters
   "kThresh": "0.0001",
   "segMinVerts": "20",
   "minPoints": "750",
   "maxPoints": "30000",
   "thinThresh": "0.05",
   "flatThresh": "0.001",
   "minLength": "0.02",
   "maxLength": "1"
  },
  "sceneId": "...",  // id of segmented scene
  "segIndices": [1,1,1,1,3,3,15,15,15,15],  // per-vertex index of mesh segment
}
```

- **Aggregated semantic annotation file (`*.aggregation.json`)**:
```javascript
{
  "sceneId": "...",  // id of annotated scene
  "appId": "...", // id + version of the tool used to create the annotation
  "segGroups": [
    {
      "id": 0,
      "objectId": 0,
      "segments": [1,4,3],
      "label": "couch"
    },
  ],
  "segmentsFile": "..." // id of the *.segs.json segmentation file referenced
}
```
[BenchmarkScripts/util_3d.py](BenchmarkScripts/util_3d.py) gives examples to parsing the semantic instance information from the `*.segs.json`, `*.aggregation.json`, and `*_vh_clean_2.ply` mesh file, with example semantic segmentation visualization in [BenchmarkScripts/3d_helpers/visualize_labels_on_mesh.py](BenchmarkScripts/3d_helpers/visualize_labels_on_mesh.py).

- **2d annotation projections (`*_2d-label.zip`, `*_2d-instance.zip`, `*_2d-label-filt.zip`, `*_2d-instance-filt.zip`)**:
Projection of 3d aggregated annotation of a scan into its RGB-D frames, according to the computed camera trajectory.